In [1]:
import os
import sys
root_dir = '/home/hanshis/workspace/LongContextInfer'
sys.path.append(root_dir)

print(f"added {root_dir} to sys.path")

import torch
from transformers import AutoTokenizer
from termcolor import colored

from models.modeling_llama_torch import LlamaForCausalLM
from models.cache_utils import SimpleCache, EvictStreamLLMCache, StreamLLMCache
from utils.decoding import Evict_Spec_cache
from utils.misc import print_config

draft = LlamaForCausalLM.from_pretrained("NousResearch/Yarn-Llama-2-7b-128k", torch_dtype=torch.float16, device_map="cuda:0")
draft_cache = EvictStreamLLMCache(draft, start_size=16, recent_size=1000-16)

added /home/hanshis/workspace/LongContextInfer to sys.path


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
prompt = """The Carnegie Technical Schools were founded in 1900 in Pittsburgh, Pennsylvania[19] by the Scottish-American industrialist and philanthropist Andrew Carnegie, who wrote "My heart is in the work", when he donated the funds to create the institution. Carnegie's vision was to open a vocational training school for the sons and daughters of working-class Pittsburghers, many of whom worked in his mills. Carnegie was inspired for the design of his school by the Pratt Institute in Brooklyn, New York, founded by industrialist Charles Pratt in 1887.[20] In 1912, the institution changed its name to Carnegie Institute of Technology (CIT) and began offering four-year degrees. During this time, CIT consisted of four constituent schools: the School of Fine and Applied Arts, the School of Apprentices and Journeymen, the School of Science and Technology, and the Margaret Morrison Carnegie School for Women.

The Mellon Institute of Industrial Research was founded in 1913 by banker and industrialist brothers Andrew Mellon (who went on to become U.S. Treasury Secretary) and Richard B. Mellon in honor of their father, Thomas Mellon, patriarch of the Mellon family. The Institute began as a research organization which performed work for government and industry on a contract basis and was initially established as a department within the University of Pittsburgh. In 1927, the Mellon Institute was incorporated as an independent nonprofit. In 1937, the Mellon Institute's iconic building was completed at its current location on Fifth Avenue.[21]

In 1967, with support from Paul Mellon, the Carnegie Institute of Technology merged with the Mellon Institute of Industrial Research to become Carnegie Mellon University. In 1973, Carnegie Mellon's coordinate women's college, the Margaret Morrison Carnegie College, merged its academic programs with the rest of the university.[22] The industrial research mission of the Mellon Institute survived the merger as the Carnegie Mellon Research Institute (CMRI) and continued doing work on contract to industry and government. In 2001, CMRI's programs were subsumed by other parts of the university or spun off into autonomous entities.[23]

Carnegie Mellon's 157.2 acre (63 ha) main campus is five miles (8 km) from downtown Pittsburgh, between Schenley Park and the neighborhoods of Squirrel Hill, Shadyside, and Oakland.[5] Carnegie Mellon is bordered to the west by the campus of the University of Pittsburgh. Carnegie Mellon owns 81 buildings in the Oakland and Squirrel Hill neighborhoods of Pittsburgh.

For decades, the center of student life on campus was Skibo Hall, the university's student union. Built in the 1950s, Skibo Hall's design was typical of mid-century modern architecture but was poorly equipped to deal with advances in computer and internet connectivity. The original Skibo Hall was razed in the summer of 1994 and replaced by a new student union that is fully Wi-Fi enabled. Known as the University Center, the building was dedicated in 1996. In 2014, Carnegie Mellon re-dedicated the University Center as the Cohon University Center in recognition of the eighth president of the university, Jared Cohon.[24]

A large grassy area known as "The Cut" forms the backbone of the campus, with a separate grassy area known as "The Mall" running perpendicular. The Cut was formed by filling in a ravine (hence the name) with soil from a nearby hill that was leveled to build the College of Fine Arts building.

The northwestern part of the campus (home to Hamburg Hall, Newell-Simon Hall, Smith Hall, and Gates Hillman Complex) was acquired from the United States Bureau of Mines in the 1980s.

Carnegie Mellon has been purchasing 100% renewable energy for its electricity since 2011.[25] The campus began to take shape in the Beaux-Arts architecture style of George Carnegie Palmer and Henry Hornbostel of Palmer & Hornbostel, winners of the 1904 competition to design the original institution and later the founder of what is now the Carnegie Mellon School of Architecture.[26]

There was little change to the campus between the first and second World War. A 1938 master plan by Githens and Keally suggested acquisition of new land along Forbes Avenue, but the plan was not fully implemented. The period starting with the construction of the Hall of the Arts building (former home of the Graduate School of Industrial Administration) in 1952 and ending with Wean Hall in 1971 saw the institutional change from Carnegie Institute of Technology to Carnegie Mellon University. New facilities were needed to respond to the university's growing national reputation in artificial intelligence, business, robotics and the arts. In addition, an expanding student population resulted in a need for improved facilities for student life, athletics and libraries. The campus finally expanded to Forbes Avenue from its original land along Schenley Park. A ravine long known as "The Cut" was initially filled in to campus level, joining "The Mall" as a major campus open space.
"""
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Yarn-Llama-2-7b-128k")
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(draft.device)

In [3]:
input_ids.shape

torch.Size([1, 1286])

In [4]:
import math
input_ids = input_ids[:, :1000]
iter_prefill = math.ceil(input_ids.shape[1] / 100)
for i in (range(iter_prefill)):
    draft_cache.evict(100)
    outputs_draft = draft(
        input_ids=input_ids[:, i*100:(i+1)*100].to(draft.device),
        past_key_values=draft_cache,
    )

In [5]:
a = draft_cache.key_cache[16].clone()

In [6]:
draft_cache.evict(1)
draft_cache.seq_len

999

In [7]:
draft_cache.key_cache[16][0][:,:16].shape

torch.Size([32, 16, 128])

In [8]:
draft_cache.key_cache[16][0][0][:16].shape

torch.Size([16, 128])

In [9]:
torch.allclose(a[0][i][:16], draft_cache.key_cache[16][0][i][:16])

True

In [10]:
a[0][i][17:].shape

torch.Size([983, 128])

In [11]:
draft_cache.key_cache[16][0][i][16:-1].shape

torch.Size([983, 128])

In [12]:
torch.allclose(a[0][i][17], draft_cache.key_cache[16][0][i][16])

False

In [13]:
a[0][i][17:].shape

torch.Size([983, 128])

In [14]:
for i in range(32):
    assert torch.allclose(a[0][i][:16], draft_cache.key_cache[16][0][i][:16]), f"i={i} [sink]"
    for j in range(983):
        assert torch.allclose(a[0][i][17+j], draft_cache.key_cache[16][0][i][16+j]), f"i={i}, {j} [recent]"

AssertionError: i=0, 3 [recent]

In [19]:
a[0]

tensor([[[ 4.2816e-02, -2.0355e-02,  1.7105e-02,  ..., -4.0234e-01,
           2.8418e-01, -1.6980e-01],
         [-8.8623e-01, -9.2090e-01, -5.6445e-01,  ..., -6.4355e-01,
          -7.1973e-01,  1.7451e+00],
         [-5.4883e-01, -1.3184e-01, -1.8250e-01,  ..., -9.5850e-01,
          -8.3887e-01,  1.3867e+00],
         ...,
         [-7.6660e-01,  2.6392e-01, -7.0605e-01,  ...,  5.6348e-01,
          -1.2832e+00,  5.8691e-01],
         [-8.9111e-01, -1.7236e-01, -6.1523e-01,  ...,  7.0068e-01,
          -8.4668e-01,  2.8320e-01],
         [-6.2158e-01, -3.9551e-01, -8.9307e-01,  ...,  2.0449e+00,
          -1.4971e+00,  4.2261e-01]],

        [[-3.0075e-02,  8.6670e-03,  9.3603e-04,  ..., -5.8008e-01,
          -7.1240e-01, -2.3181e-01],
         [ 5.6738e-01,  3.9575e-01, -2.9810e-01,  ..., -3.3223e+00,
           4.4849e-01,  8.5693e-01],
         [ 5.8984e-01,  1.6406e-01, -2.2964e-02,  ..., -2.5449e+00,
           2.0117e+00,  3.4160e+00],
         ...,
         [ 3.2251e-01,  1

In [ ]:
size_keep = self.recent_size - incoming
self.key_cache[layer_idx][:, :, self.start_size:-incoming] = self.key_cache[layer_idx][:, :, self.seq_len-size_keep:self.seq_len]
self.value_cache[layer_idx][:, :, self.start_size:-incoming] = self.value_cache[layer_idx][:, :, self.seq_len-size_keep:self.seq_len]

In [20]:
draft_cache.key_cache[16][0]

tensor([[[ 4.2816e-02, -2.0355e-02,  1.7105e-02,  ..., -4.0234e-01,
           2.8418e-01, -1.6980e-01],
         [-8.8623e-01, -9.2090e-01, -5.6445e-01,  ..., -6.4355e-01,
          -7.1973e-01,  1.7451e+00],
         [-5.4883e-01, -1.3184e-01, -1.8250e-01,  ..., -9.5850e-01,
          -8.3887e-01,  1.3867e+00],
         ...,
         [-8.9111e-01, -1.7236e-01, -6.1523e-01,  ...,  7.0068e-01,
          -8.4668e-01,  2.8320e-01],
         [-6.2158e-01, -3.9551e-01, -8.9307e-01,  ...,  2.0449e+00,
          -1.4971e+00,  4.2261e-01],
         [-6.2158e-01, -3.9551e-01, -8.9307e-01,  ...,  2.0449e+00,
          -1.4971e+00,  4.2261e-01]],

        [[-3.0075e-02,  8.6670e-03,  9.3603e-04,  ..., -5.8008e-01,
          -7.1240e-01, -2.3181e-01],
         [ 5.6738e-01,  3.9575e-01, -2.9810e-01,  ..., -3.3223e+00,
           4.4849e-01,  8.5693e-01],
         [ 5.8984e-01,  1.6406e-01, -2.2964e-02,  ..., -2.5449e+00,
           2.0117e+00,  3.4160e+00],
         ...,
         [ 1.4294e-01,  6

In [5]:
from utils.sampling import norm_logits, sample
from utils.misc import spec_stream
next_token = sample(norm_logits(outputs_draft.logits[:,-1,:], temperature=0.8 ,top_k=-1, top_p=0.9))

with torch.inference_mode():
    for i in range(1000):
        draft_cache.evict(1)
        outputs_draft = draft(
            input_ids=next_token.to(draft.device),
            past_key_values=draft_cache,
        )
        assert outputs_draft.logits.shape == (1, 1, 32000)
        next_token = sample(norm_logits(outputs_draft.logits[:,-1,:], temperature=0.8 ,top_k=-1, top_p=0.9))
        spec_stream(next_token[0], tokenizer)

In the early  1 9 6 0 s , the university added the Scott Hall , a student residence , as well as the Miller Hall , the first building of the new business school . The College of Fine Arts was also given a new home , the Hill man Library . The growth of the university ' s schools of computer science , information systems , and industrial management led to the construction of the William Howard Hall and the We an Building . A major addition to the campus in the  1 9 8 0 s was the We an Science Building . 
 
 The next campus master plan , " C arn eg ie M ell on University : A Master Plan ", was developed by the C M U Office of Pl anning and Fac ilities and approved by the Board of Trust ees in  2 0 0 . It includes  3 1 acres ,  8 5 % of the total campus a cre age , and a planned increase in total en roll ment from  8 , 0 0 0 to  1 5 , 0 0 . 
 
 In  2 0 1 0 , Car neg ie M ell on began work on a  1 4 - acre project to re develop land along For bes Avenue between More wood Avenue and P ink e

In [26]:
a = draft_cache.key_cache[0].clone()
a.shape

torch.Size([1, 32, 1000, 128])

In [ ]:
# create grid search scripts

draft_list = ['llama-68M-align']
gamma_list = [4,5,6,7,8]
chunk_size_list = [2,3,4,5,6,8,10,12,15,16,24,32,48,64]
budget_list = [0.05, 0.0475, 0.045, 0.0425, 0.04, 0.0375, 0.035, 0.033, 0.03]

for draft in draft_list:
    for gamma in gamma_list:
        for chunk_size in chunk_size_list:
            for budget in budget_list:
                template = f"CUDA_VISIBLE_DEVICES=0 python test/e2e_graph_chain_retrieval.py --log_csv --draft {draft} --gamma {gamma} --chunk_size {chunk_size} --budget {budget} --prefill 122880 --dataset gs"
                print(template)

In [8]:
import pandas as pd

# 替换为你的CSV文件路径
csv_file_path = '/home/hanshis/workspace/LongContextInfer/test/report/A100_E2E_graph_chain_retrieval.csv'

# 读取CSV文件
df = pd.read_csv(csv_file_path)
# df = df[df['budget'] == 0.033]
# 对'a', 'b', 'c', 'd'列组成的组合进行分组，并计算每组'e'的平均值
result = df.groupby(['budget','chunk_size','gamma'])['token/s'].mean().reset_index().sort_values(by='token/s', ascending=False)
result

,budget,chunk_size,gamma,token/s
151,0.0350,6,6,42.333985
538,0.0500,8,6,42.316488
545,0.0500,10,8,42.214505
365,0.0425,16,6,41.994353
366,0.0425,16,7,41.961542
...,...,...,...,...
3,0.0300,2,7,33.547048
261,0.0400,2,8,33.476774
68,0.0330,2,7,33.365905
69,0.0330,2,8,31.407150
